In [2]:
import pandas as pd
import requests
from openpyxl import load_workbook
from openpyxl import Workbook
import re

# 주소록에서 주소만 가져오기
filePath = r'C:\\sqlite\\mysql\\code\\AI\\Mid_project\\data\\all.xlsx'
df_from_excel = pd.read_excel(filePath, engine='openpyxl')
# df_from_excel.columns = df_from_excel.loc[0].tolist()
# df_from_excel = df_from_excel.drop(index=list(range(0, 5)))

# print(df_from_excel)
# 오픈API 사용준비
url = 'http://api.vworld.kr/req/address?'
params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type='
road_type = 'ROAD'  # 도로명 주소
road_type2 = 'PARCEL'  # 지번 주소
address = '&address='
keys = '&key='
primary_key = '683E2ED2-8E65-3440-BA7F-F66E58D3044D'

# # 좌표 얻어오는 함수


def request_geo(road):
    page = requests.get(url+params+road_type+address+road+keys+primary_key)
    json_data = page.json()
    if json_data['response']['status'] == 'OK':
        x = json_data['response']['result']['point']['x']
        y = json_data['response']['result']['point']['y']
        return x, y
    else:
        x = 0
        y = 0
        return x, y


# 얻어 온 좌표 엑셀로 저장
try:
    wb = load_workbook(
        r'C:\\sqlite\\mysql\\code\\AI\\Mid_project\\data\\좌표.xlsx', data_only=True)
    sheet = wb.active
except:
    wb = Workbook()
    sheet = wb.active

address_list = df_from_excel['시군구'].to_list()


for num, value in enumerate(address_list):
    # 주소에서 괄호 빼기(subtract)
    addr = re.sub(r'\([^)]*\)', '', value)
    print(addr)
    # 좌표 얻어오기
    x, y = request_geo(addr)
    # 엑셀에 써 넣기
    # sheet.append([address_list[num], addr, x, y])
    sheet.append([addr, x, y])


wb.save(r"C:\\sqlite\\mysql\\code\\AI\\total좌표.xlsx")

서울특별시 중랑구 면목동
서울특별시 노원구 하계동
서울특별시 송파구 문정동
서울특별시 관악구 신림동
서울특별시 성북구 장위동
서울특별시 양천구 신정동
서울특별시 구로구 신도림동
서울특별시 강남구 삼성동
서울특별시 송파구 풍납동
서울특별시 중랑구 면목동
서울특별시 강북구 번동
서울특별시 동대문구 장안동
서울특별시 동대문구 전농동
서울특별시 중랑구 묵동
서울특별시 강남구 대치동
서울특별시 중랑구 신내동
서울특별시 강동구 성내동
서울특별시 구로구 오류동
서울특별시 동대문구 전농동
서울특별시 종로구 누하동
서울특별시 서초구 서초동
서울특별시 서초구 반포동
서울특별시 동작구 상도1동
서울특별시 관악구 신림동
서울특별시 노원구 상계동
서울특별시 강북구 미아동
서울특별시 영등포구 당산동
서울특별시 은평구 대조동
서울특별시 강서구 화곡동
서울특별시 구로구 궁동
서울특별시 은평구 응암동
서울특별시 성북구 삼선동3가
서울특별시 서대문구 남가좌동
서울특별시 강서구 방화동
서울특별시 동작구 사당동
서울특별시 서초구 서초동
서울특별시 강동구 명일동
서울특별시 강북구 번동
서울특별시 강동구 암사동
서울특별시 노원구 중계동
서울특별시 영등포구 여의도동
서울특별시 은평구 진관동
서울특별시 강남구 청담동
서울특별시 마포구 상암동
서울특별시 강남구 대치동
서울특별시 동대문구 제기동
서울특별시 송파구 신천동
서울특별시 양천구 신월동
서울특별시 도봉구 창동
서울특별시 마포구 망원동
서울특별시 강남구 논현동
서울특별시 서초구 내곡동
서울특별시 송파구 송파동
서울특별시 서대문구 연희동
서울특별시 서대문구 북가좌동
서울특별시 용산구 한강로3가
서울특별시 마포구 망원동
서울특별시 중구 태평로2가
서울특별시 광진구 중곡동
서울특별시 영등포구 대림동
서울특별시 광진구 구의동
서울특별시 성동구 성수동1가
서울특별시 강서구 화곡동
서울특별시 동대문구 청량리동
서울특별시 동작구 사당동
서울특별시 광진구 구의동
서울특별시 구로구 구로동
서울특별시 동대문구 전농동
서울특별시 중랑구 면목동
서울특별시 동작구 상도동


In [2]:
# import folium
# import pandas as pd
# filePath = r'C:\\sqlite\\mysql\\code\\AI\\Mid_project\\data\\보호구역외사고_좌표.xlsx'
# df_from_excel = pd.read_excel(filePath, engine='openpyxl', header=None)
# df_from_excel.columns = ['주소','x','y']

# # print(df_from_excel)
# name_list = df_from_excel.iloc[:, 0].tolist()
# px_list = df_from_excel.iloc[:, 1].tolist()
# py_list = df_from_excel.iloc[:, 2].tolist()
# map = folium.Map(location=[37, 127], zoom_start=7)

# for i in range(len(name_list)):
#     if px_list[i]!= 0:
#         print(px_list[i], py_list[i])
#         marker = folium.Marker([py_list[i], px_list[i]], popup=name_list[i], icon=folium.Icon(color='red'))
#         marker.add_to(map)

# map.show_in_browser()

127.263398039 37.008455647
127.154490765 37.29924151
126.783103753 37.50153198
126.797289024 37.432482435
126.804487396 37.345039109
127.064418946 37.728701279
127.170544263 37.442152508
127.14269524 37.473448128
126.766975736 37.490874344
127.072802388 37.131752891
127.269642408 37.005662907
126.783859992 37.466418608
127.210155975 37.009039636
126.804487396 37.345039109
127.126679942 37.461835522
126.966864104 37.385472304
127.024604685 37.257752207
126.861074276 37.453973169
126.781785829 37.435427816
127.072802388 37.131752891
127.053230577 37.890882556
126.851027701 37.296374427
126.781785829 37.435427816
127.456352108 37.282921336
126.753266157 37.501732632
127.044017154 37.905571054
126.779829804 37.672451345
127.11898126 37.296764416
127.242345723 37.329416064
127.133066079 37.433773898
127.065068188 37.157576479
127.096510171 37.007874378
126.836760821 37.638139628
126.726935433 37.797877109
127.134654601 37.000297276
127.12796322 37.366794771
127.11898126 37.296764416
126.721